# CAT BOOST

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [2]:
labels = pd.read_csv('../../csv/train_labels.csv')
labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [3]:
values = pd.read_csv('../../csv/train_values.csv')
values.T

,0,1,2,3,4,5,6,7,8,9,...,260591,260592,260593,260594,260595,260596,260597,260598,260599,260600
building_id,802906,28830,94947,590882,201944,333020,728451,475515,441126,989500,...,560805,207683,226421,159555,827012,688636,669485,602512,151409,747594
geo_level_1_id,6,8,21,22,11,8,9,20,0,26,...,20,10,8,27,8,25,17,17,26,21
geo_level_2_id,487,900,363,418,131,558,475,323,757,886,...,368,1382,767,181,268,1335,715,51,39,9
geo_level_3_id,12198,2812,8973,10694,1488,6089,12066,12236,7219,994,...,5980,1903,8613,1537,4718,1621,2060,8163,1851,9101
count_floors_pre_eq,2,2,2,2,3,2,2,2,2,1,...,1,2,2,6,2,1,2,3,2,3
age,30,10,10,10,30,10,25,0,15,0,...,25,25,5,0,20,55,0,55,10,10
area_percentage,6,8,5,6,8,9,3,8,8,13,...,5,5,13,13,8,6,6,6,14,7
height_percentage,5,7,5,5,9,5,4,6,6,4,...,3,5,5,12,5,3,5,7,6,6
land_surface_condition,t,o,t,t,t,t,n,t,t,t,...,n,t,t,t,t,n,t,t,t,n
foundation_type,r,r,r,r,r,r,r,w,r,i,...,r,r,r,r,r,r,r,r,r,r


In [4]:
values["building_id"].count() == values["building_id"].drop_duplicates().count()

True

In [5]:
to_be_categorized = ["land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for row in to_be_categorized:
    values[row] = values[row].astype("category")

In [6]:
datatypes = dict(values.dtypes)
for row in values.columns:
    if datatypes[row] != "int64" and datatypes[row] != "int32" and \
       datatypes[row] != "int16" and datatypes[row] != "int8":
        continue
    if values[row].nlargest(1).item() > 32767 and values[row].nlargest(1).item() < 2**31:
        values[row] = values[row].astype(np.int32)
    elif values[row].nlargest(1).item() > 127:
        values[row] = values[row].astype(np.int16)
    else:
        values[row] = values[row].astype(np.int8)

In [7]:
labels["building_id"] = labels["building_id"].astype(np.int32)
labels["damage_grade"] = labels["damage_grade"].astype(np.int8)

In [8]:
important_values = values\
                .merge(labels, on="building_id")
important_values.drop(columns=["building_id"], inplace = True)
important_values["geo_level_1_id"] = important_values["geo_level_1_id"].astype("category")
important_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,2
260597,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
260598,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,3
260599,26,39,1851,2,10,14,6,t,r,x,...,0,0,0,0,0,0,0,0,0,2


In [9]:
X_train, X_test, y_train, y_test = train_test_split(important_values.drop(columns = 'damage_grade'),
                                                    important_values['damage_grade'], test_size = 0.2, random_state = 123)

In [10]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    X_train = encode_and_bind(X_train, feature)
    X_test = encode_and_bind(X_test, feature)

In [11]:
pip install catboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
from catboost import CatBoostClassifier

In [16]:
model = CatBoostClassifier( iterations = 5000,
                            eval_metric='Accuracy',
                            loss_function='MultiClass',
                            use_best_model=True,
                            random_seed=1,
                            max_depth = None,
                            learning_rate = 0.15,
                            boost_from_average = False,
                            verbose=True)
model.fit(X_train, y_train, 
          cat_features=['geo_level_2_id', 'geo_level_3_id'],
          eval_set=(X_test,y_test))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.6930305	test: 0.6952284	best: 0.6952284 (0)	total: 100ms	remaining: 8m 20s
1:	learn: 0.6974242	test: 0.6969743	best: 0.6969743 (1)	total: 191ms	remaining: 7m 57s
2:	learn: 0.7017076	test: 0.7013872	best: 0.7013872 (2)	total: 263ms	remaining: 7m 18s
3:	learn: 0.7021105	test: 0.7020779	best: 0.7020779 (3)	total: 363ms	remaining: 7m 33s
4:	learn: 0.7024415	test: 0.7022313	best: 0.7022313 (4)	total: 455ms	remaining: 7m 34s
5:	learn: 0.7037989	test: 0.7041308	best: 0.7041308 (5)	total: 538ms	remaining: 7m 27s
6:	learn: 0.7044944	test: 0.7045145	best: 0.7045145 (6)	total: 646ms	remaining: 7m 40s
7:	learn: 0.7046623	test: 0.7045529	best: 0.7045529 (7)	total: 730ms	remaining: 7m 35s
8:	learn: 0.7049213	test: 0.7045721	best: 0.7045721 (8)	total: 821ms	remaining: 7m 35s
9:	learn: 0.7050796	test: 0.7049941	best: 0.7049941 (9)	total: 950ms	remaining: 7m 54s
10:	learn: 0.7054490	test: 0.7055697	best: 0.7055697 (10)	total: 1.06s	remaining: 8m 2s
11:	learn: 0.7086627	test: 0.7095221	best:

93:	learn: 0.7362625	test: 0.7377832	best: 0.7377832 (93)	total: 10.6s	remaining: 9m 12s
94:	learn: 0.7363008	test: 0.7377449	best: 0.7377832 (93)	total: 10.7s	remaining: 9m 11s
95:	learn: 0.7363056	test: 0.7375914	best: 0.7377832 (93)	total: 10.8s	remaining: 9m 12s
96:	learn: 0.7364735	test: 0.7376873	best: 0.7377832 (93)	total: 11s	remaining: 9m 13s
97:	learn: 0.7365838	test: 0.7375722	best: 0.7377832 (93)	total: 11.1s	remaining: 9m 15s
98:	learn: 0.7365359	test: 0.7374187	best: 0.7377832 (93)	total: 11.2s	remaining: 9m 14s
99:	learn: 0.7365695	test: 0.7375146	best: 0.7377832 (93)	total: 11.3s	remaining: 9m 14s
100:	learn: 0.7366798	test: 0.7376873	best: 0.7377832 (93)	total: 11.4s	remaining: 9m 15s
101:	learn: 0.7366702	test: 0.7376106	best: 0.7377832 (93)	total: 11.6s	remaining: 9m 14s
102:	learn: 0.7366894	test: 0.7375338	best: 0.7377832 (93)	total: 11.7s	remaining: 9m 15s
103:	learn: 0.7367805	test: 0.7376106	best: 0.7377832 (93)	total: 11.8s	remaining: 9m 15s
104:	learn: 0.73691

185:	learn: 0.7410495	test: 0.7415053	best: 0.7415053 (185)	total: 22.6s	remaining: 9m 44s
186:	learn: 0.7412030	test: 0.7414862	best: 0.7415053 (185)	total: 22.7s	remaining: 9m 43s
187:	learn: 0.7411598	test: 0.7414286	best: 0.7415053 (185)	total: 22.8s	remaining: 9m 43s
188:	learn: 0.7412989	test: 0.7414862	best: 0.7415053 (185)	total: 22.9s	remaining: 9m 43s
189:	learn: 0.7413853	test: 0.7414286	best: 0.7415053 (185)	total: 23.1s	remaining: 9m 43s
190:	learn: 0.7413565	test: 0.7415053	best: 0.7415053 (185)	total: 23.2s	remaining: 9m 44s
191:	learn: 0.7414428	test: 0.7415821	best: 0.7415821 (191)	total: 23.4s	remaining: 9m 46s
192:	learn: 0.7414716	test: 0.7415437	best: 0.7415821 (191)	total: 23.6s	remaining: 9m 47s
193:	learn: 0.7415675	test: 0.7417548	best: 0.7417548 (193)	total: 23.7s	remaining: 9m 48s
194:	learn: 0.7415723	test: 0.7417548	best: 0.7417548 (193)	total: 23.9s	remaining: 9m 49s
195:	learn: 0.7415915	test: 0.7416396	best: 0.7417548 (193)	total: 24s	remaining: 9m 49s
1

277:	learn: 0.7443640	test: 0.7426181	best: 0.7429059 (276)	total: 34.6s	remaining: 9m 48s
278:	learn: 0.7443832	test: 0.7426181	best: 0.7429059 (276)	total: 34.7s	remaining: 9m 47s
279:	learn: 0.7444551	test: 0.7425798	best: 0.7429059 (276)	total: 34.9s	remaining: 9m 47s
280:	learn: 0.7444887	test: 0.7425030	best: 0.7429059 (276)	total: 35s	remaining: 9m 47s
281:	learn: 0.7445127	test: 0.7426181	best: 0.7429059 (276)	total: 35.1s	remaining: 9m 47s
282:	learn: 0.7445750	test: 0.7429059	best: 0.7429059 (276)	total: 35.3s	remaining: 9m 47s
283:	learn: 0.7445990	test: 0.7428676	best: 0.7429059 (276)	total: 35.4s	remaining: 9m 47s
284:	learn: 0.7445894	test: 0.7428867	best: 0.7429059 (276)	total: 35.5s	remaining: 9m 47s
285:	learn: 0.7445271	test: 0.7428100	best: 0.7429059 (276)	total: 35.7s	remaining: 9m 47s
286:	learn: 0.7445942	test: 0.7428484	best: 0.7429059 (276)	total: 35.8s	remaining: 9m 47s
287:	learn: 0.7446134	test: 0.7429443	best: 0.7429443 (287)	total: 36s	remaining: 9m 48s
288

...

4620:	learn: 0.8009737	test: 0.7468583	best: 0.7480478 (3978)	total: 10m 55s	remaining: 53.8s
4621:	learn: 0.8009353	test: 0.7469734	best: 0.7480478 (3978)	total: 10m 56s	remaining: 53.7s
4622:	learn: 0.8009497	test: 0.7469350	best: 0.7480478 (3978)	total: 10m 56s	remaining: 53.5s
4623:	learn: 0.8009977	test: 0.7469158	best: 0.7480478 (3978)	total: 10m 56s	remaining: 53.4s
4624:	learn: 0.8009641	test: 0.7469734	best: 0.7480478 (3978)	total: 10m 56s	remaining: 53.2s
4625:	learn: 0.8010121	test: 0.7469734	best: 0.7480478 (3978)	total: 10m 56s	remaining: 53.1s
4626:	learn: 0.8009929	test: 0.7469350	best: 0.7480478 (3978)	total: 10m 57s	remaining: 53s
4627:	learn: 0.8010313	test: 0.7469926	best: 0.7480478 (3978)	total: 10m 57s	remaining: 52.8s
4628:	learn: 0.8010505	test: 0.7470118	best: 0.7480478 (3978)	total: 10m 57s	remaining: 52.7s
4629:	learn: 0.8010121	test: 0.7469542	best: 0.7480478 (3978)	total: 10m 57s	remaining: 52.6s
4630:	learn: 0.8010361	test: 0.7469734	best: 0.7480478 (3978)	

4709:	learn: 0.8019091	test: 0.7470118	best: 0.7480478 (3978)	total: 11m 11s	remaining: 41.4s
4710:	learn: 0.8019043	test: 0.7470885	best: 0.7480478 (3978)	total: 11m 11s	remaining: 41.2s
4711:	learn: 0.8018851	test: 0.7471844	best: 0.7480478 (3978)	total: 11m 12s	remaining: 41.1s
4712:	learn: 0.8018323	test: 0.7471269	best: 0.7480478 (3978)	total: 11m 12s	remaining: 40.9s
4713:	learn: 0.8018899	test: 0.7471653	best: 0.7480478 (3978)	total: 11m 12s	remaining: 40.8s
4714:	learn: 0.8018611	test: 0.7471077	best: 0.7480478 (3978)	total: 11m 12s	remaining: 40.7s
4715:	learn: 0.8019091	test: 0.7470309	best: 0.7480478 (3978)	total: 11m 12s	remaining: 40.5s
4716:	learn: 0.8019954	test: 0.7470501	best: 0.7480478 (3978)	total: 11m 13s	remaining: 40.4s
4717:	learn: 0.8020482	test: 0.7471653	best: 0.7480478 (3978)	total: 11m 13s	remaining: 40.2s
4718:	learn: 0.8020434	test: 0.7472228	best: 0.7480478 (3978)	total: 11m 13s	remaining: 40.1s
4719:	learn: 0.8021105	test: 0.7472036	best: 0.7480478 (3978

4798:	learn: 0.8027197	test: 0.7472036	best: 0.7480478 (3978)	total: 11m 25s	remaining: 28.7s
4799:	learn: 0.8027005	test: 0.7471844	best: 0.7480478 (3978)	total: 11m 25s	remaining: 28.6s
4800:	learn: 0.8027868	test: 0.7471653	best: 0.7480478 (3978)	total: 11m 25s	remaining: 28.4s
4801:	learn: 0.8028108	test: 0.7471461	best: 0.7480478 (3978)	total: 11m 26s	remaining: 28.3s
4802:	learn: 0.8028396	test: 0.7471461	best: 0.7480478 (3978)	total: 11m 26s	remaining: 28.1s
4803:	learn: 0.8028492	test: 0.7470693	best: 0.7480478 (3978)	total: 11m 26s	remaining: 28s
4804:	learn: 0.8028444	test: 0.7470693	best: 0.7480478 (3978)	total: 11m 26s	remaining: 27.9s
4805:	learn: 0.8028204	test: 0.7470501	best: 0.7480478 (3978)	total: 11m 26s	remaining: 27.7s
4806:	learn: 0.8028444	test: 0.7470693	best: 0.7480478 (3978)	total: 11m 26s	remaining: 27.6s
4807:	learn: 0.8028540	test: 0.7470118	best: 0.7480478 (3978)	total: 11m 26s	remaining: 27.4s
4808:	learn: 0.8028924	test: 0.7470118	best: 0.7480478 (3978)	

4887:	learn: 0.8035687	test: 0.7469158	best: 0.7480478 (3978)	total: 11m 40s	remaining: 16.1s
4888:	learn: 0.8035447	test: 0.7469542	best: 0.7480478 (3978)	total: 11m 40s	remaining: 15.9s
4889:	learn: 0.8035879	test: 0.7469926	best: 0.7480478 (3978)	total: 11m 40s	remaining: 15.8s
4890:	learn: 0.8035831	test: 0.7468775	best: 0.7480478 (3978)	total: 11m 41s	remaining: 15.6s
4891:	learn: 0.8035831	test: 0.7469158	best: 0.7480478 (3978)	total: 11m 41s	remaining: 15.5s
4892:	learn: 0.8036215	test: 0.7469158	best: 0.7480478 (3978)	total: 11m 41s	remaining: 15.3s
4893:	learn: 0.8036598	test: 0.7468966	best: 0.7480478 (3978)	total: 11m 41s	remaining: 15.2s
4894:	learn: 0.8036838	test: 0.7469542	best: 0.7480478 (3978)	total: 11m 41s	remaining: 15.1s
4895:	learn: 0.8036982	test: 0.7468583	best: 0.7480478 (3978)	total: 11m 41s	remaining: 14.9s
4896:	learn: 0.8037174	test: 0.7468966	best: 0.7480478 (3978)	total: 11m 42s	remaining: 14.8s
4897:	learn: 0.8037366	test: 0.7468199	best: 0.7480478 (3978

4975:	learn: 0.8042882	test: 0.7469542	best: 0.7480478 (3978)	total: 11m 57s	remaining: 3.46s
4976:	learn: 0.8043361	test: 0.7469734	best: 0.7480478 (3978)	total: 11m 57s	remaining: 3.31s
4977:	learn: 0.8043361	test: 0.7469734	best: 0.7480478 (3978)	total: 11m 57s	remaining: 3.17s
4978:	learn: 0.8043793	test: 0.7469350	best: 0.7480478 (3978)	total: 11m 57s	remaining: 3.03s
4979:	learn: 0.8043697	test: 0.7469734	best: 0.7480478 (3978)	total: 11m 57s	remaining: 2.88s
4980:	learn: 0.8043361	test: 0.7468966	best: 0.7480478 (3978)	total: 11m 57s	remaining: 2.74s
4981:	learn: 0.8043266	test: 0.7468391	best: 0.7480478 (3978)	total: 11m 58s	remaining: 2.59s
4982:	learn: 0.8042930	test: 0.7468966	best: 0.7480478 (3978)	total: 11m 58s	remaining: 2.45s
4983:	learn: 0.8042882	test: 0.7468966	best: 0.7480478 (3978)	total: 11m 58s	remaining: 2.31s
4984:	learn: 0.8043074	test: 0.7468583	best: 0.7480478 (3978)	total: 11m 58s	remaining: 2.16s
4985:	learn: 0.8042978	test: 0.7469734	best: 0.7480478 (3978

In [17]:
model_pred = model.predict(X_test)
f1_score(y_test, model_pred, average='micro')

0.74804781182249

In [18]:
test_values = pd.read_csv('../../csv/test_values.csv', index_col = "building_id")

In [19]:
test_values_subset = test_values
test_values_subset["geo_level_1_id"] = test_values_subset["geo_level_1_id"].astype("category")

In [20]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    test_values_subset = encode_and_bind(test_values_subset, feature)

In [21]:
preds = model.predict(test_values_subset)

In [22]:
submission_format = pd.read_csv('../../csv/submission_format.csv', index_col = "building_id")

In [23]:
my_submission = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [24]:
my_submission.to_csv('../../csv/predictions/jf/cat-boost/jf-model-2-prueba-submission.csv')